### Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación

### Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones 2021
Búsqueda y Recomendación para Textos Legales

Mentor: Jorge E. Pérez Villella

# Práctico Análisis y Visualización

Integrantes:

### Objetivos:

* Generar un corpus con todos los documentos. 

* Dividir el corpus en tokens, graficar el histograma de frecuencia de palabras demostrando la ley Zipf. 

* Analizar palabras más frecuentes y menos frecuentes. Seleccionar 5 documentos de cada fuero y realizar el mismo análisis. ¿Se repiten las palabras? 

* Hacer lo mismo con n-gramas.

* Visualizar la frecuencia de palabras en una nube de palabras.

* Elaborar una breve conclusión de lo encontrado

Fecha de Entrega: 6 de junio de 2021

# Generar un corpus con todos los documentos

In [1]:
#!conda install -y -c anaconda spacy 

In [1]:
#!python -m spacy download es_core_news_sm

In [2]:
!conda install -y -c conda-forge dask 

/Users/coviedo/opt/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Solving environment: done

## Package Plan ##

  environment location: /Users/coviedo/opt/anaconda3/envs/diplodatos-ayvd

  added / updated specs:
    - dask


The following packages will be UPDATED:

  ca-certificates    anaconda::ca-certificates-2020.10.14-0 --> conda-forge::ca-certificates-2020.12.5-h033912b_0
  certifi                anaconda::certifi-2020.6.20-py36_0 --> conda-forge::certifi-2020.12.5-py36h79c6626_1


Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [14]:
import spacy
from spacy.lang.es.examples import sentences
import glob
import os
import pandas as pd 
import shutil

from dask.distributed import Client, progress


#client = Client(n_workers=4, threads_per_worker=2, memory_limit='1GB')
#client

In [15]:
import dask
import dask.dataframe as dd

In [1]:
root_path = "Documentos"

directories = [x[0] for x in os.walk(root_path)]

directories.pop(0)

for directory in directories:
    print (directory)


NameError: name 'os' is not defined

In [20]:

def generate_corpus_df(directories):
   corpus = []

    for directory in directories:
      
        file_list = glob.glob(os.path.join(os.getcwd(), directory, "*.txt"))


        for file_path in file_list:
            with open(file_path) as f_input:
                corpus.append([f_input.read() , directory])

    return dd.from_pandas(data = pd.DataFrame(corpus, columns=["text", "classifier"] ), npartitions=4 )
    #return pd.DataFrame(corpus, columns=["text", "classifier"] )
   

In [21]:
corpus_df = generate_corpus_df(directories)

In [31]:
corpus_df.persist()

,text,classifier
npartitions=4,,
0,object,object
63,...,...
126,...,...
189,...,...
250,...,...


In [25]:
def generate_corpus_file(directory, corpus_file_name): 
    
    outfilename = os.path.join(os.getcwd(), directory , corpus_file_name )
    input_files = os.path.join(os.getcwd(), directory , '*.txt' )
    
    with open(outfilename, 'wb') as outfile:
        for filename in glob.glob(input_files):
            if filename == outfilename:
                # don't want to copy the output into the output
                continue
            with open(filename, 'rb') as readfile:
                shutil.copyfileobj(readfile, outfile)

In [26]:
# Generamos el corpus de cada fuero en el archivo corpust.txt. Hacemos esto puesto que es muchos menos 
# costoso que pasar a un unico string los valores del data frame

corpus_file_name = "corpus.txt"
for directory in directories:
    generate_corpus_file(directory, corpus_file_name)

In [27]:
def generate_agregate_df_corpus(directories, corpus_file_name): 
    
    agregate_df_corpus = pd.DataFrame()
        
    for directory in directories: 
        
        corpus_file = os.path.join(os.getcwd(), directory , corpus_file_name ) 
        print(corpus_file)
        with open(corpus_file, 'r') as file:
            data = file.read().replace('\n', ' ')
            print(data[:10])
            agregate_df_corpus.append({'text' : data, 'tag' : directory } , ignore_index = True)
            print ("Se agrego")
    
    return agregate_df_corpus


In [28]:

def generate_agregate_df_corpus_2(directories, corpus_file_name): 
    
    agregate_df_corpus = pd.DataFrame(columns=['text', 'tag'])
        
    for directory in directories: 
        
        corpus_file = os.path.join(os.getcwd(), directory , corpus_file_name ) 
        data = open(corpus_file, 'r').read()
        agregate_df_corpus = agregate_df_corpus.append({'text' : data, 'tag' : directory } , ignore_index = True)
        
    return agregate_df_corpus



In [25]:
agregate_df_corpus = generate_agregate_df_corpus_2(directories ,corpus_file_name )

In [26]:
agregate_df_corpus[:]

,text,tag
0,DATOS DE LA CAUSA\nSede: Ciudad de Córdoba. \n...,Documentos/MENORES
1,SALA PENAL - TRIBUNAL SUPERIOR\n\nProtocolo de...,Documentos/PENAL
2,"AUTO NÚMERO: sesenta y seis \nCórdoba, cinco d...",Documentos/FAMILIA
3,SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...,Documentos/LABORAL


# Dividir el corpus en tokens, graficar el histograma de frecuencia de palabras demostrando la ley Zipf. 

In [12]:
spacy_nlp = spacy.load("es_core_news_sm")

# no funciona, consume mucha memoria
def generate_corpus_by_tag(df, tag):
    
    corpus = corpus_df[corpus_df['classifier'] == classifier ].text.to_list()

    return corpus

# no funciona, consume mucha memoria
spacy_nlp.max_length = 3000000
for directory in directories:
    print (f"Generando corpus de {directory}")
    corpus = "".join(corpus_df[corpus_df['classifier'] == classifier ].text.to_list())
    corpus_doc = spacy_nlp(corpus)
    

In [13]:
spacy_nlp.max_length = 7000000
doc = spacy_nlp(agregate_df_corpus[agregate_df_corpus['tag'] == 'Documentos/MENORES'].text[0])

In [14]:
words = [token.text
         for token in doc
         if not token.is_stop and not token.is_punct]

In [15]:
from collections import Counter
word_freq = Counter(words)

In [16]:
common_words = word_freq.most_common(10)

In [17]:
common_words

[(' ', 11660),
 ('y', 10122),
 ('a', 8780),
 ('\n\n', 2170),
 ('fs', 2010),
 ('niño', 1666),
 ('\n', 1410),
 ('o', 1158),
 ('Sra.', 1156),
 ('art', 1106)]

In [ ]:
doc_penal = spacy_nlp(agregate_df_corpus[agregate_df_corpus['tag'] == 'Documentos/PENAL'].text[1])

In [ ]:
words = [token.text
         for token in doc
         if not token.is_stop and not token.is_punct]

In [ ]:
common_words = word_freq.most_common(10)